In [ ]:
Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, canada
Para esta tarea, se le pedirá que explore y agrupe los vecindarios en Toronto.

Empiece por crear un nuevo Notebook para esta tarea.